## Download de dependêcias

In [11]:
!pip install datasets transformers accelerate -q

## Baixar as bibliotecas

In [12]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import pandas as pd
import torch
import random
import numpy as np
from datasets import Dataset

## Separação treino e teste

In [13]:

"""""
Este script realiza a preparação de dados para análise de sentimento, definindo um ambiente controlado com sementes aleatórias para garantir reprodutibilidade. 
Ele carrega dados de sentimentos classificados, ajusta as etiquetas para um formato binário e divide os dados em conjuntos de treinamento e teste.

Variáveis:
    seed_val (int): Valor da semente para garantir a reprodutibilidade.
    device (torch.device): Define o dispositivo de cálculo ('cuda' ou 'cpu') dependendo da disponibilidade de CUDA.
    df (DataFrame): DataFrame contendo os dados carregados do arquivo CSV.
    dataset (Dataset): Conjunto de dados formatado para uso em modelos de aprendizado de máquina.
    train_test_dataset (dict): Dicionário contendo os subconjuntos de dados de treino e teste.
    train_dataset (Dataset): Subconjunto de dados para treinamento.
    valid_dataset (Dataset): Subconjunto de dados para validação/teste.
"""""
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)    
torch.manual_seed(seed_val)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
df = pd.read_csv("../data/processed_classification-labeled.csv")
df['label'] = df['sentiment'].apply(lambda x: 0 if x in [0, 1] else 1)
dataset = Dataset.from_pandas(df)
train_test_dataset = dataset.train_test_split(test_size=0.2, seed=seed_val)
train_dataset = train_test_dataset['train']
valid_dataset = train_test_dataset['test']

## Tokenização do dataframe

In [14]:
"""
Este módulo configura e aplica tokenização a conjuntos de dados de treino e validação usando um tokenizer pré-treinado especificamente para análise de sentimentos em tweets.

Funções:
    tokenize_function: Recebe exemplos de texto e os tokeniza usando o tokenizer especificado, com um comprimento máximo de 128 tokens. 
    A função garante que todos os textos sejam padronizados para o mesmo comprimento através do preenchimento e truncamento.

Variáveis:
    model_checkpoint (str): Identificador do modelo BERT pré-treinado, hospedado no Hugging Face Model Hub, otimizado para análise de sentimentos em tweets.
    tokenizer: Instância do tokenizer carregada do modelo especificado pelo `model_checkpoint`.

Processamento:
    - O tokenizer é aplicado aos conjuntos de dados de treino e validação utilizando a função `tokenize_function`, processando os dados em lotes para maior eficiência.
"""

model_checkpoint = "finiteautomata/bertweet-base-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def tokenize_function(examples):
    return tokenizer(examples['comment'], padding="max_length", truncation=True, max_length=128)
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Map:   0%|          | 0/2300 [00:00<?, ? examples/s]

Map:   0%|          | 0/576 [00:00<?, ? examples/s]

## Configuração do modelo

In [15]:
"""
Este módulo configura um modelo de classificação de sequências para análise de sentimentos e define uma função para computar métricas de desempenho do modelo.

Modelo:
    - Inicializa um modelo de AutoModelForSequenceClassification do Hugging Face transformers, configurado para classificação binária.
    - Carrega o modelo pré-treinado especificado por `model_checkpoint`, ajustando-o para classificar duas classes (positiva e negativa).
    - O modelo é configurado para ignorar tamanhos de entrada incompatíveis e é enviado ao dispositivo de cálculo especificado (CPU ou GPU).

Funções:
    - compute_metrics(pred): Calcula várias métricas de desempenho a partir das predições do modelo, incluindo precisão, recall, f1-score, e matriz de confusão para ambas as classes (negativa e não-negativa).

Variáveis:
    - model: Instância do modelo de classificação de sequência configurada para análise de sentimentos.
    - device: Dispositivo de hardware (CPU ou GPU) para onde o modelo é enviado para realizar os cálculos.
"""


model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, ignore_mismatched_sizes=True).to(device)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1])
    acc = accuracy_score(labels, preds)
    conf_matrix = confusion_matrix(labels, preds)

    # Convertendo a matriz de confusão para lista
    conf_matrix_str = np.array2string(conf_matrix)
    return {
        'accuracy': acc,
        'f1_negativos': f1[1],
        'f1_nao_negativos': f1[0],
        'recall_negativos': recall[1],
        'recall_nao_negativos': recall[0],
        'precision_negativos': precision[1],
        'precision_nao_negativos': precision[0],
        'confusion_matrix': conf_matrix_str
    }





Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Treinamento do modelo

In [16]:

"""
Este módulo configura e executa o treinamento de um modelo de classificação de sequências para análise de sentimentos usando a biblioteca transformers.

Configuração:
    - Define os argumentos de treinamento usando `TrainingArguments`, especificando diretórios de saída, estratégias de avaliação e salvamento, taxa de aprendizado, tamanho dos lotes para treino e avaliação, número de épocas de treinamento, decaimento de peso, e o critério para selecionar o melhor modelo.

Treinador:
    - Inicializa um `Trainer` com o modelo, argumentos de treinamento, conjuntos de dados de treino e validação, e a função de computação de métricas definida anteriormente.
    - Executa o treinamento do modelo utilizando o método `.train()`.

Variáveis:
    - training_args (TrainingArguments): Configuração dos parâmetros de treinamento, incluindo opções para otimização e gerenciamento de modelo.
    - trainer (Trainer): Instância responsável por gerenciar e executar o treinamento do modelo.

Processamento:
    - O treinamento é realizado de acordo com as configurações especificadas, visando a maximização da acurácia. O modelo com melhor desempenho é automaticamente carregado ao final do treinamento.
"""


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Negativos,F1 Nao Negativos,Recall Negativos,Recall Nao Negativos,Precision Negativos,Precision Nao Negativos,Confusion Matrix
1,No log,0.415660,0.821181,0.876647,0.675079,0.905941,0.622093,0.849188,0.737931,[[107 65] [ 38 366]]
2,0.442300,0.403297,0.826389,0.880096,0.685535,0.908416,0.633721,0.853488,0.746575,[[109 63] [ 37 367]]


Trainer is attempting to log a value of "[[107  65]
 [ 38 366]]" of type <class 'str'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[109  63]
 [ 37 367]]" of type <class 'str'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=576, training_loss=0.43403201301892597, metrics={'train_runtime': 7802.841, 'train_samples_per_second': 0.59, 'train_steps_per_second': 0.074, 'total_flos': 302577713664000.0, 'train_loss': 0.43403201301892597, 'epoch': 2.0})

O modelo BERT final demonstrou melhorias significativas em várias métricas de avaliação entre as duas épocas. A acurácia, por exemplo, teve um aumento modesto, indicando que o modelo está classificando corretamente uma maior proporção das amostras de teste. Além disso, houve melhorias nos valores de F1 Score para sentimentos negativos e não-negativos, o que é fundamental em conjuntos de dados com desequilíbrio de classes. A precisão e a revocação para as classes também melhoraram, o que sugere uma capacidade aprimorada do modelo em identificar corretamente as classes minoritárias, sem um aumento significativo nos falsos positivos.

A matriz de confusão mostra uma boa distinção entre as classes, e as perdas de treinamento e validação diminuíram, indicando que o modelo está convergindo bem e generalizando eficazmente para novos dados. Dadas estas melhorias contínuas e o equilíbrio entre precisão e revocação, o modelo é considerado adequado para implantação final, oferecendo robustez e eficácia na tarefa de análise de sentimentos, onde precisão e generalização são cruciais.